Based on: https://huggingface.co/docs/transformers/quicktour#pipeline-usage

In [ ]:
import pandas as pd

from inspect import signature
from pyspark.sql.functions import col, pandas_udf
from sparkext.huggingface import model_udf
from transformers import pipeline

In [ ]:
pipe = pipeline("text-classification")

In [ ]:
pipe("What can I say that hasn't been said already. I think this place is totally worth the hype.")

In [ ]:
pipe("I will not say much about this film, because there is not much to say, because there is not much there to talk about.")

## Inference using Spark DL UDF

In [ ]:
from sparkext.huggingface import model_udf

In [ ]:
# only use first sentence of IMDB reviews
@pandas_udf("string")
def first_sentence(text: pd.Series) -> pd.Series:
    return pd.Series([s.split(".")[0] for s in text])

df = spark.read.parquet("imdb_test").withColumn("sentence", first_sentence(col("lines"))).select("sentence")
df.take(10)

In [ ]:
# note: need to manually specify return_type per pipe output above
classify = model_udf(pipe, return_type="label string, score float")

In [ ]:
predictions = df.withColumn("preds", classify(col("sentence"))).select("sentence", "preds.*")

In [ ]:
predictions.show()